<a href="https://colab.research.google.com/github/marcomachado/teo-me-why-web-scrapping/blob/main/tabnews_collect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import datetime
import json
import os
import time

In [88]:
os.system(f'rm -rf data')

0

In [89]:
class Collector:

  def __init__(self, url, instance_name):
    self.url = url
    self.instance_name = instance_name

    self.path_json = f'data/{self.instance_name}/contents/json/'
    self.path_parquet = f'data/{self.instance_name}/contents/parquet/'

    self.create_folders()

  def create_folders(self):

    if os.path.exists(self.path_json) == False:
      os.makedirs(self.path_json)

    if os.path.exists(self.path_parquet) == False:
      os.makedirs(self.path_parquet)

  def get_response(self, **kwargs):
    resp = requests.get(self.url, params=kwargs)
    return resp

  def save_json(self, data):
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S.%f")

    with open(f'{self.path_json}{now}.json', 'w') as open_file:
      json.dump(data, open_file, indent=4)
      open_file.close()

  def save_parquet(self, data):
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S.%f")

    df = pd.DataFrame(data)
    df.to_parquet(f'{self.path_parquet}{now}.parquet', index=False)

  def save_data(self, data, option='json'):
    if option == 'json':
      self.save_json(data)
    elif option =='parquet':
      self.save_parquet(data)


  def get_and_save(self, save_format='json', **kwargs):
    resp = self.get_response(**kwargs)
    if resp.status_code == 200:
      data = resp.json()
      self.save_data(data, save_format)
    else:
      data = None
      print(f'Request sem sucesso: {resp.status_code} {self.url}')

    return data

  def auto_exec(self, save_format='json', data_stop='2000-01-01'):
    page = 1
    while True:
      print(page)
      data = self.get_and_save(save_format=save_format,
                        page=page,
                        per_page=100,
                        strategy='new')

      if data == None:
        print('Erro ao coletar dados')
        time.sleep()
      else:
        date_last = pd.to_datetime(data[-1]['updated_at']).date()
        if date_last < pd.to_datetime(data_stop).date() or len(data) < 100:
          break

      page += 1
      time.sleep(2)


In [90]:
url = 'https://www.tabnews.com.br/api/v1/contents'
collector = Collector(url, 'tabnews')
collector.auto_exec(data_stop='2024-03-01')

1
2
3
4
5
